# Tutorial for Dual-color mesoscopic imaging of both ACh and calcium across the neocortex of awake mice.

This tutorial shows how to access the data related to [DANDI:001172](https://dandiarchive.org/dandiset/001172/draft) using `pynwb`. 

Contents:

- [Streaming an NWB file](#stream-nwb)
- [Reading an NWB file](#read-nwb)
- [Access Subject and Session metadata](#access-subject)
- [Access Raw Imaging](#access-raw-imaging)
- [Access Processed Imaging](#access-raw-imaging)
- [Access Processed Behavior](#access-facemap)
- [Access Visual Stimuli](#access-visual)
- [Access Wheel Signal](#access-wheel)

# Select the subject and session you want to load

In [ ]:
subject_id = "grabAM05"
session_id = "11252019-vis-stim"
nwbfile_path = f"sub-{subject_id}/sub-{subject_id}_ses-{session_id}_behavior+image+ophys.nwb"

# Streaming an NWB file <a id="stream-nwb"></a>

This section demonstrates how to access the files on the [DANDI Archive](https://dandiarchive.org) without downloading them. Based on the [Streaming NWB files](https://pynwb.readthedocs.io/en/stable/tutorials/advanced_io/streaming.html) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

The `dandi.dandiapi.DandiAPIClient` can be used to get the S3 URL of the NWB file stored in the DANDI Archive.

In [ ]:
from dandi.dandiapi import DandiAPIClient

dandiset_id = "001172"

with DandiAPIClient() as client:
    client.dandi_authenticate() #This line is necessary because the dataset is in embargoed mode and only owners can view the data, once it will be published this line can be removed.
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(nwbfile_path)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)


We will use `remfile` for streaming the file. You can read more about `remfile` at [this tutorial section](https://pynwb.readthedocs.io/en/stable/tutorials/advanced_io/streaming.html#method-3-remfile).

In [ ]:
import h5py
import remfile

file = remfile.File(s3_url)
h5_file = h5py.File(file, "r")

# Reading an NWB file <a name="read-nwb"></a>

This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO
io = NWBHDF5IO(file=h5_file, load_namespaces=True)
nwbfile = io.read()
nwbfile

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from all_sessions.Session datajoint table.

In [ ]:
nwbfile.session_start_time

The experiment description or the session description can be easily access with `nwbfile.experiment_description` and `nwbfile.session_description` 

In [ ]:
print(nwbfile.experiment_description)

In [ ]:
print(nwbfile.session_description)

# Access subject <a name="access-subject"></a>

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

# Access OnePhoton Imaging

This section demonstrates how to access the raw One Photon imaging data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw OnePhoton Imaging <a name="access-raw-imaging"></a>

The raw OnePhoton imaging data is stored in `pynwb.ophys.OnePhotonSeries` objects (for each channel and plane separately) which is added to `nwbfile.acquisition`.

The data in [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
photon_series_names = [photon_series_name for photon_series_name in nwbfile.acquisition.keys() if "OnePhotonSeries" in photon_series_name]
print(photon_series_names)

In [ ]:
# Visualize the imaging data.
from matplotlib import pyplot as plt
fig, axs = plt.subplots(nrows=len(photon_series_names), ncols=1, sharex=True, sharey=True)
for ps_index,ps_name in enumerate(photon_series_names):    
    photon_series = nwbfile.acquisition[ps_name]
    axs[ps_index].imshow(photon_series.data[70], cmap="binary")
    axs[ps_index].set_title(ps_name.replace("_"," "))
plt.show()


Imaging metadata include information on the **device** used to acquire the image and the **optical channel** specs contained in the imaging plane object

In [ ]:
nwbfile.devices["OnePhotonMicroscope"]

In [ ]:
imaging_planes_names = [imaging_planes_name for imaging_planes_name in nwbfile.imaging_planes.keys()]
print(imaging_planes_names)
nwbfile.imaging_planes[imaging_planes_names[0]]

## Accessing the segmentation data

The segmentation output for the Two Photon Imaging data is stored in `nwbfile.processing["ophys"]`. 

In NWB, the [PlaneSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.PlaneSegmentation) class stores the detected regions of interest in the [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) data. The [ImageSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImageSegmentation) can contain multiple `PlaneSegmentation` tables, so that we can store results of different segmentation algorithms or different segmentation classes.

We can access the plane segmentation for the [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) data as 
`nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentationChannel_number_Plane_number_"]`.


In [ ]:
nwbfile.processing["ophys"]

## Access DF/F one photon series

In [ ]:
dff_photon_series_names = [dff_photon_series_name for dff_photon_series_name in nwbfile.processing["ophys"].data_interfaces if "DFFOnePhotonSeries" in dff_photon_series_name]
print(dff_photon_series_names)

In [ ]:
import numpy as np
fig, axs = plt.subplots(nrows=len(dff_photon_series_names), ncols=1, sharex=True, sharey=True)
for ps_index,ps_name in enumerate(dff_photon_series_names):    
    dff_photon_series = nwbfile.processing["ophys"][ps_name]
    data = dff_photon_series.data[:]
    data[data == 0] = np.nan
    axs[ps_index].imshow(data[70], vmin=-0.01, cmap="binary")
    axs[ps_index].set_title(ps_name.replace("_"," "))
plt.show()

#### Plot Image Segmentation: Functional Parcellation

In [ ]:
plane_segmentation_names = list(nwbfile.processing["ophys"]["ImageSegmentation"].plane_segmentations.keys())
_ = [print(name) for name in nwbfile.processing["ophys"]["ImageSegmentation"].plane_segmentations.keys()]

In [ ]:
import numpy as np
from matplotlib.colors import ListedColormap
import random

photon_series = nwbfile.acquisition["OnePhotonSeriesBlueExcitationGreenChannel"]
plane_seg_name = plane_segmentation_names[0]
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True)   
plane_segmentation = nwbfile.processing["ophys"]["ImageSegmentation"].plane_segmentations[plane_seg_name]
for image_mask in plane_segmentation.image_mask:
    random_color = [random.random() for _ in range(3)] + [1]
    custom_cmap = ListedColormap([[0, 0, 0, 0], random_color])
    ax.imshow(image_mask.T, aspect="auto", cmap=custom_cmap)
ax.set_title(plane_seg_name)
plt.show()

## Visualize fluorescence traces
The fluorescence traces are stored in a [Fluorescence](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.Fluorescence) container, the raw traces can be accessed as `nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"]`.

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

fluorescence_traces_names = [name for name in nwbfile.processing["ophys"]["Fluorescence"].roi_response_series.keys()]

fluorescence_traces_name = fluorescence_traces_names[0]
start_index = 0
stop_index = -1

fluorescence_rate = nwbfile.processing["ophys"]["Fluorescence"][fluorescence_traces_name].rate
fluorescence_starting_time = nwbfile.processing["ophys"]["Fluorescence"][fluorescence_traces_name].starting_time
start_time =fluorescence_starting_time + start_index*fluorescence_rate
stop_time = fluorescence_starting_time + stop_index*fluorescence_rate

fluorescence_traces = nwbfile.processing["ophys"]["Fluorescence"][fluorescence_traces_name].data[:, :10]

df = pd.DataFrame(fluorescence_traces)
df["time"] = np.linspace(start_time, stop_time, fluorescence_traces.shape[0])
df.set_index("time", inplace=True)
df.columns.name = 'ROIs'

import plotly.express as px

fig = px.line(df, facet_row="ROIs", facet_row_spacing=0.01)

# hide and lock down axes
fig.update_xaxes(visible=True, fixedrange=False)
fig.update_yaxes(visible=False, fixedrange=False)

# remove facet/subplot labels
fig.update_layout(annotations=[], overwrite=True)

# strip down the rest of the plot
fig.update_layout(
    showlegend=True,
    plot_bgcolor="white",
    margin=dict(t=10, l=10, b=10, r=10)
)

fig.show(config=dict(displayModeBar=True))

# Access Processed Behavior <a name="access-facemap"></a>

This section demonstrates how to access the face motion components, the eye tracking and pupil tracking data.

In [ ]:
nwbfile.processing["behavior"]["EyeTracking"]

In [ ]:
nwbfile.processing["behavior"]["PupilTracking"]

In [ ]:
start_index = 0
stop_index = 2000

pupil_area_timestamps = nwbfile.processing["behavior"]["PupilTracking"]["pupil_area"].timestamps[start_index:stop_index]
pupil_area_signal = nwbfile.processing["behavior"]["PupilTracking"]["pupil_area"].data[start_index:stop_index]

fig, ax = plt.subplots(figsize=(6, 2), dpi=300)
ax.plot(pupil_area_timestamps, pupil_area_signal, color="cyan", linewidth=0.5, label="Pupil Area")

# Hide top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Create the 
ax.legend(frameon=False, bbox_to_anchor=(.95, 1), loc='upper left', prop={'size': 8})

ax.tick_params(axis='y', labelsize=8)
plt.xlabel('Time (s)', fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.show()

In [ ]:
motion_svd_series = nwbfile.processing["behavior"]["MotionSVDSeriesWhiskers"]
motion_svd_series

In [ ]:
motion_svd_masks = nwbfile.processing["behavior"]["MotionSVDMasksWhiskers"]
motion_svd_masks

In [ ]:
n_components_to_visualize = 5
start_index = 0
stop_index = 100
fig, ax = plt.subplots(nrows=n_components_to_visualize, ncols=2, dpi=300)
for i in range(n_components_to_visualize):
    mask =np.array(motion_svd_masks[i].image_mask[i])
    ax[i, 0].imshow(mask, cmap="binary")
    ax[i, 0].set_axis_off()
    series = motion_svd_series.data[start_index:stop_index,i]
    timestamps = motion_svd_series.timestamps[start_index:stop_index]
    ax[i,1].plot(timestamps, series)

# Access Visual Stimulus, Wheel Activation or Airpuff tables <a name="access-visual"></a>

This section demonstrates how to access the visual stimulus or wheel activation or airpuff data.


In [ ]:
table_object_type = "VisualStimulus" # "VisualStimulus" "Airpuff" "WheelActivation"
table_object = nwbfile.intervals[table_object_type].to_dataframe()
table_object

In [ ]:
start_index = 0
stop_index = 1000

fluorescence_rate = nwbfile.processing["ophys"]["Fluorescence"][fluorescence_traces_name].rate
fluorescence_starting_time = nwbfile.processing["ophys"]["Fluorescence"][fluorescence_traces_name].starting_time
start_time =fluorescence_starting_time + start_index/fluorescence_rate
stop_time = fluorescence_starting_time + stop_index/fluorescence_rate

ROI_index=22
fluorescence_data = nwbfile.processing["ophys"]["Fluorescence"][fluorescence_traces_name].data[:, ROI_index]
fluorescence_timestamps = np.linspace(start_time, stop_time, fluorescence_data.shape[0])

fig, ax = plt.subplots(figsize=(6, 2), dpi=300)
line_handle, = ax.plot(fluorescence_timestamps, fluorescence_data, color="green", linewidth=0.5, label=f"ROI {ROI_index}".format(ROI_index=ROI_index))

handles = []
for (stimulus_start, stimulus_stop) in zip(table_object["start_time"], table_object["stop_time"]):
    handle = ax.fill_between(fluorescence_timestamps, np.nanmax(fluorescence_data), where=(fluorescence_timestamps >= stimulus_start) & (fluorescence_timestamps <= stimulus_stop),color='yellow', edgecolor='none', alpha=0.9, label=table_object_type)
    handles.append(handle)

# Hide top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Create the legend with the appropriate handles
handles = [line_handle, handles[0]]
labels = [f"ROI {ROI_index}".format(ROI_index=ROI_index), table_object_type]
ax.legend(handles, labels, frameon=False, bbox_to_anchor=(.95, 1), loc='upper left', prop={'size': 8})

ax.tick_params(axis='y', labelsize=8)
plt.xlabel('Time (s)', fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.show()

# Access Wheel Signal and Wheel Speed <a name="access-wheel"></a>
This section demonstrates how to access the raw wheel signal acquired with Spike2 and the processed wheel speed trace.

In [ ]:
nwbfile.acquisition["WheelSignal"]

In [ ]:
start_index = 0
stop_index = 200000

wheel_signal_rate = nwbfile.acquisition["WheelSignal"].rate
wheel_signal_starting_time = nwbfile.acquisition["WheelSignal"].starting_time
start_time = wheel_signal_starting_time + start_index/wheel_signal_rate
stop_time = wheel_signal_starting_time + stop_index/wheel_signal_rate

wheel_signal = nwbfile.acquisition["WheelSignal"].data[start_index:stop_index]
wheel_signal_timestamps = np.linspace(start_time, stop_time, wheel_signal.shape[0])

fig, ax = plt.subplots(figsize=(6, 2), dpi=300)
ax.plot(wheel_signal_timestamps, wheel_signal, color="red", linewidth=0.5, label="Wheel Signal")

# Hide top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Create the 
ax.legend(frameon=False, bbox_to_anchor=(.95, 1), loc='upper left', prop={'size': 8})

ax.tick_params(axis='y', labelsize=8)
plt.xlabel('Time (s)', fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.show()

In [ ]:
nwbfile.processing["behavior"]["WheelSpeed"]

In [ ]:
start_index = 0
stop_index = 200000

wheel_speed_rate = nwbfile.processing["behavior"]["WheelSpeed"].rate
wheel_speed_starting_time = nwbfile.processing["behavior"]["WheelSpeed"].starting_time
start_time = wheel_speed_starting_time + start_index/wheel_speed_rate
stop_time = wheel_speed_starting_time + stop_index/wheel_speed_rate

wheel_speed = nwbfile.processing["behavior"]["WheelSpeed"].data[start_index:stop_index]
wheel_speed_timestamps = np.linspace(start_time, stop_time, wheel_speed.shape[0])

fig, ax = plt.subplots(figsize=(6, 2), dpi=300)
ax.plot(wheel_speed_timestamps, wheel_speed, color="red", linewidth=0.5, label="Wheel Speed")

# Hide top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Create the 
ax.legend(frameon=False, bbox_to_anchor=(.95, 1), loc='upper left', prop={'size': 8})

ax.tick_params(axis='y', labelsize=8)
plt.xlabel('Time (s)', fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.show()